# `hmmlearn`

Does not seem to work with `scikit-learn` 0.2.

    conda install -c omnia hmmlearn=0.3.0b=np112py36_0
    conda install -c omnia scikit-learn=0.19.1=py36hedc7406_0


In [50]:
import random
import numpy as np
import pandas as pd

In [51]:
no_accel_samples = 100
window_accel_samples = 10
acceleration_model_function = np.random.random

In [1]:
import numpy as np
from hmmlearn import hmm

In [73]:
np.random.seed(42)

n_components = 6

model = hmm.GaussianHMM(n_components=n_components
                        , covariance_type="full")

# Start probabilities
p_start_rum_wel = 0.6-0.01
p_start_eat_wel = 0.3-0.01
p_start_oth_wel = 0.1-0.01
p_start_rum_ill = 0.01
p_start_eat_ill = 0.01
p_start_oth_ill = 0.01

# Probability delta-in-transition from well to ill
p_d_wel_to_ill = 0.01
p_d_ill_to_well = 0.1

# Probability of transition from rumination
p_trans_rum_to_rum = 0.7
p_trans_rum_to_eat = 0.2
p_trans_rum_to_oth = 0.1

# Probability of transition from eating
p_trans_eat_to_rum = 0.3
p_trans_eat_to_eat = 0.5
p_trans_eat_to_oth = 0.2

# Probability of transition from other
p_trans_oth_to_rum = 0.3
p_trans_oth_to_eat = 0.3
p_trans_oth_to_oth = 0.4

# Prob of transition from rumination
p_trans_rum_wel_to_rum_wel = p_trans_rum_to_rum-p_d_wel_to_ill
p_trans_rum_wel_to_eat_wel = p_trans_rum_to_eat-p_d_wel_to_ill
p_trans_rum_wel_to_oth_wel = p_trans_rum_to_oth-p_d_wel_to_ill
p_trans_rum_wel_to_rum_ill = p_d_wel_to_ill
p_trans_rum_wel_to_eat_ill = p_d_wel_to_ill
p_trans_rum_wel_to_oth_ill = p_d_wel_to_ill

# Prob of transition from eating
p_trans_eat_wel_to_rum_wel = p_trans_eat_to_rum-p_d_wel_to_ill
p_trans_eat_wel_to_eat_wel = p_trans_eat_to_eat-p_d_wel_to_ill
p_trans_eat_wel_to_oth_wel = p_trans_eat_to_oth-p_d_wel_to_ill
p_trans_eat_wel_to_rum_ill = p_d_wel_to_ill
p_trans_eat_wel_to_eat_ill = p_d_wel_to_ill
p_trans_eat_wel_to_oth_ill = p_d_wel_to_ill

# Prob of transition from other
p_trans_oth_wel_to_rum_wel = p_trans_oth_to_rum-p_d_wel_to_ill
p_trans_oth_wel_to_eat_wel = p_trans_oth_to_eat-p_d_wel_to_ill
p_trans_oth_wel_to_oth_wel = p_trans_oth_to_oth-p_d_wel_to_ill
p_trans_oth_wel_to_rum_ill = p_d_wel_to_ill
p_trans_oth_wel_to_eat_ill = p_d_wel_to_ill
p_trans_oth_wel_to_oth_ill = p_d_wel_to_ill

# SICK
# Prob of transition from rumination sick
p_trans_rum_ill_to_rum_wel = p_d_ill_to_well
p_trans_rum_ill_to_eat_wel = p_d_ill_to_well
p_trans_rum_ill_to_oth_wel = p_d_ill_to_well
p_trans_rum_ill_to_rum_ill = p_trans_rum_to_rum - p_d_ill_to_well
p_trans_rum_ill_to_eat_ill = p_trans_rum_to_eat - p_d_ill_to_well
p_trans_rum_ill_to_oth_ill = p_trans_rum_to_oth - p_d_ill_to_well

# Prob of transition from eating sick
p_trans_eat_ill_to_rum_wel = p_d_ill_to_well
p_trans_eat_ill_to_eat_wel = p_d_ill_to_well
p_trans_eat_ill_to_oth_wel = p_d_ill_to_well
p_trans_eat_ill_to_rum_ill = p_trans_eat_to_rum - p_d_ill_to_well
p_trans_eat_ill_to_eat_ill = p_trans_eat_to_eat - p_d_ill_to_well
p_trans_eat_ill_to_oth_ill = p_trans_eat_to_oth - p_d_ill_to_well

# Prob of transition from other sick
p_trans_oth_ill_to_rum_wel = p_d_ill_to_well
p_trans_oth_ill_to_eat_wel = p_d_ill_to_well
p_trans_oth_ill_to_oth_wel = p_d_ill_to_well
p_trans_oth_ill_to_rum_ill = p_trans_oth_to_rum - p_d_ill_to_well
p_trans_oth_ill_to_eat_ill = p_trans_oth_to_eat - p_d_ill_to_well
p_trans_oth_ill_to_oth_ill = p_trans_oth_to_oth - p_d_ill_to_well

# Setting start probabilities 
model.startprob_ = np.array([p_start_rum_wel
                             , p_start_eat_wel
                             , p_start_oth_wel
                             , p_start_rum_ill
                             , p_start_eat_ill
                             , p_start_oth_ill])

model.transmat_ = np.array([[p_trans_rum_wel_to_rum_wel # 0 to
                             , p_trans_rum_wel_to_eat_wel # 0 to
                             , p_trans_rum_wel_to_oth_wel # 0 to
                             , p_trans_rum_wel_to_rum_ill # 0 to
                             , p_trans_rum_wel_to_eat_ill # 0 to
                             , p_trans_rum_wel_to_oth_ill], # 0 to
                            [p_trans_eat_wel_to_rum_wel # 1
                             , p_trans_eat_wel_to_eat_wel # 1
                             , p_trans_eat_wel_to_oth_wel # 1
                             , p_trans_eat_wel_to_rum_ill # 1
                             , p_trans_eat_wel_to_eat_ill # 1
                             , p_trans_eat_wel_to_oth_ill], # 1
                            [p_trans_oth_wel_to_rum_wel # 2
                             , p_trans_oth_wel_to_eat_wel # 2
                             , p_trans_oth_wel_to_oth_wel # 2
                             , p_trans_oth_wel_to_rum_ill # 2
                             , p_trans_oth_wel_to_eat_ill # 2
                             , p_trans_oth_wel_to_oth_ill], # 2
                            # Ill ruminating to 
                            [p_trans_rum_ill_to_rum_wel # 3
                             , p_trans_rum_ill_to_eat_wel # 3
                             , p_trans_rum_ill_to_oth_wel # 3
                             , p_trans_rum_ill_to_rum_ill # 3
                             , p_trans_rum_ill_to_eat_ill # 3
                             , p_trans_rum_ill_to_oth_ill], # 3
                            # Ill eating to
                            [p_trans_eat_ill_to_rum_wel # 4
                             , p_trans_eat_ill_to_eat_wel # 4
                             , p_trans_eat_ill_to_oth_wel # 4
                             , p_trans_eat_ill_to_rum_ill # 4
                             , p_trans_eat_ill_to_eat_ill # 4
                             , p_trans_eat_ill_to_oth_ill], # 4
                            # 5 Ill other to 
                            [p_trans_oth_ill_to_rum_wel # 5 
                             , p_trans_oth_ill_to_eat_wel # 5
                             , p_trans_oth_ill_to_oth_wel # 5
                             , p_trans_oth_ill_to_rum_ill # 5
                             , p_trans_oth_ill_to_eat_ill # 5
                             , p_trans_oth_ill_to_oth_ill] # 5
                           ])

model.means_ = np.array([
    [0.0, 0.0]    # 0 
    , [3.0, -3.0] # 1
    , [5.0, 10.0] # 2
    , [0.0, 0.0]  # 3
    , [3.0, -3.0] # 4
    , [5.0, 10.0] # 5

])

model.covars_ = np.tile(np.identity(2), (n_components, 1, 1))

X, Z = model.sample(200)

In [74]:
Z

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 0, 0, 0, 0, 1, 0, 1, 2, 0,
       1, 1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2, 0, 1, 1, 1, 1, 3, 2, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 2, 2, 1, 2, 1, 2, 2, 0, 0, 1, 1, 1, 1, 1,
       3, 3, 3, 3, 4, 5, 5, 4, 4, 5, 5, 5, 5, 5, 5, 5, 3, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 2, 1, 0, 1,
       0, 1, 1, 2, 2, 2, 1, 0, 2, 1, 1, 2, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 1, 0, 5, 5, 4, 4, 4, 3, 0, 0, 0, 2, 2, 2, 1, 0, 5, 5, 1, 0, 0,
       0, 1, 2, 0, 1, 0, 0, 0, 0, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 4, 5, 2, 2, 0, 0, 0,
       0, 0])

In [5]:
X

array([[-1.11188012e+00,  3.18902185e-01],
       [ 1.57921282e+00,  7.67434729e-01],
       [-5.80878134e-01, -5.25169807e-01],
       [ 2.41962272e-01, -1.91328024e+00],
       [ 8.16445081e-01, -1.52387600e+00],
       [-9.08024076e-01, -1.41230370e+00],
       [-6.29474961e-01,  5.97720467e-01],
       [-5.44382725e-01,  1.10922590e-01],
       [-6.00253850e-01,  9.47439821e-01],
       [-6.01706612e-01,  1.85227818e+00],
       [-5.33648804e-01, -5.52786232e-03],
       [ 5.82254491e+00,  8.77915635e+00],
       [ 5.77831304e+00, -1.80636028e+00],
       [ 7.38466580e-01,  1.71368281e-01],
       [-1.00908534e+00, -1.58329421e+00],
       [-4.60638771e-01,  1.05712223e+00],
       [-1.13055230e+00,  1.34428883e-01],
       [ 2.32307800e+00, -2.38832371e+00],
       [ 5.82122795e-01,  8.87748460e-01],
       [ 2.16078248e+00, -3.30921238e+00],
       [ 3.49184671e+00,  1.10996470e+01],
       [-1.10633497e+00, -1.19620662e+00],
       [ 3.83479542e+00, -2.70343862e+00],
       [ 3.

In [52]:
def accelerometer(
    no_accel_samples
    , acceleration_model_function):
    
    return pd.DataFrame(
        {'acceleration':
             acceleration_model_function(
                 number_of_accelerometer_samples)})

In [53]:
def features(acceleration_df
             , window_accel_samples
             , feature_function):
    
    length = len(acceleration_df)
    grouper = np.arange(length, dtype=int)//window_accel_samples
    
    return acceleration_df.groupby(grouper).apply(feature_function)

In [54]:
a_df = accelerometer(no_accel_samples=no_accel_samples
    , acceleration_model_function=acceleration_model_function)

f_df = features(acceleration_df=a_df
             ,window_accel_samples=window_accel_samples
             ,feature_function=sum)

In [56]:
import hmmlearn